In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0

In [3]:
Q.<alpha> = NumberField(xx^2+3)

In [4]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [5]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [6]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [7]:
line = Line([y, z])

In [8]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [9]:
%time SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : alpha*c})

CPU times: user 5.12 s, sys: 41.9 ms, total: 5.16 s
Wall time: 5.17 s


In [10]:
%time adm_SE9 = SE9.find_admissible_projectivities()

CPU times: user 3.97 s, sys: 294 ms, total: 4.26 s
Wall time: 53.7 s


In [11]:
len(adm_SE9)

1296

In [12]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)

CPU times: user 2.34 s, sys: 132 ms, total: 2.48 s
Wall time: 9.6 s


In [13]:
len(simm_SE9)

54

In [16]:
perms = []
for el in simm_SE9:
    perms.append(Permutation(SE9.apply_proj_to_eck(el)).to_cycles())

In [17]:
PermutationGroup(perms).order()

18

In [18]:
PermutationGroup(perms).structure_description()

'(C3 x C3) : C2'

In [19]:
#lines

In [20]:
keys = list(SE9.cl_lines.keys())

In [22]:
perms_lines = []
for simm in simm_SE9:
    perms_lines.append(Permutation(SE9.apply_proj_to_lines(simm)).to_cycles())

In [23]:
Gr = PermutationGroup(perms_lines)

In [24]:
Gr.order()

54

In [25]:
Gr.structure_description()

'((C3 x C3) : C3) : C2'

In [26]:
eck_labels = SE9.eckardt_points_labels
lines_three_eck = []
for i in range(7):
    for j in range(i+1, 8):
        for k in range(j+1, 9):
            point1 = SE9.eckardt_points[i]
            point2 = SE9.eckardt_points[j]
            point3 = SE9.eckardt_points[k]   
            if matrix([point1, point2, point3]).minors(3) == [0,0,0,0]:
                print(eck_labels[i], eck_labels[j], eck_labels[k])
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                planes = get_two_planes_containing_line(line12_pl)
                lines_three_eck.append(Line(planes,[point1, point2], line12_pl))

['E1', 'G4', 'F14'] ['E2', 'G3', 'F23'] ['F13', 'F24', 'F56']
['E1', 'G4', 'F14'] ['E3', 'G5', 'F35'] ['F15', 'F26', 'F34']
['E1', 'G4', 'F14'] ['E4', 'G6', 'F46'] ['E6', 'G1', 'F16']
['E1', 'G4', 'F14'] ['E5', 'G2', 'F25'] ['F12', 'F36', 'F45']
['E2', 'G3', 'F23'] ['E3', 'G5', 'F35'] ['E5', 'G2', 'F25']
['E2', 'G3', 'F23'] ['E4', 'G6', 'F46'] ['F15', 'F26', 'F34']
['E2', 'G3', 'F23'] ['E6', 'G1', 'F16'] ['F12', 'F36', 'F45']
['E3', 'G5', 'F35'] ['E4', 'G6', 'F46'] ['F12', 'F36', 'F45']
['E3', 'G5', 'F35'] ['E6', 'G1', 'F16'] ['F13', 'F24', 'F56']
['E4', 'G6', 'F46'] ['E5', 'G2', 'F25'] ['F13', 'F24', 'F56']
['E5', 'G2', 'F25'] ['E6', 'G1', 'F16'] ['F15', 'F26', 'F34']
['F12', 'F36', 'F45'] ['F13', 'F24', 'F56'] ['F15', 'F26', 'F34']


In [27]:
[[line1.are_incident(line2) for line2 in lines_three_eck] for line1 in lines_three_eck]

[[True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True]]

In [28]:
lines_three_eck[0].get_plane_containing_another_incident_line(lines_three_eck[1])

x + (-1/4*alpha - 1/4)*y + (1/4*alpha + 1/4)*z